In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim

In [2]:
def Create_Slim_DateSet(dataset_dir,num_samples):
    _ITEMS_TO_DESCRIPTIONS = {
        'image': 'A [32 x 32 x 3]  image.',
        'label': 'A single integer between 0 and 8',
    }
    reader = tf.TFRecordReader
    # extrct key
    keys_to_features = {
      'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
      'image/format': tf.FixedLenFeature((), tf.string, default_value='png'),
      'image/class/label': tf.FixedLenFeature(
          [], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
    }
    # decoder
    items_to_handlers = {
     'image': slim.tfexample_decoder.Image(shape=[32, 32, 3]),
     'label': slim.tfexample_decoder.Tensor('image/class/label', shape=[]),
    }
    decoder = slim.tfexample_decoder.TFExampleDecoder(
         keys_to_features, items_to_handlers)

    labels_to_names = None

    dataset_out = slim.dataset.Dataset(
         data_sources = dataset_dir,
         reader = reader,
         decoder = decoder,
         num_samples = num_samples,
         num_classes = 5,
         items_to_descriptions=_ITEMS_TO_DESCRIPTIONS,
         labels_to_names=labels_to_names)
    return dataset_out

In [3]:
def load_batch(dataset, batch_size=32):
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32,
        common_queue_min=8)    
    image_raw, label = data_provider.get(['image', 'label'])
    images_raw, labels = tf.train.shuffle_batch(
        [image_raw, label], batch_size=batch_size, num_threads=4,
        capacity=1000 + 3 * batch_size,
        min_after_dequeue=1000)
    
    return images_raw, labels

In [4]:
import tensorflow as tf
import tensorflow.contrib.slim.nets 
vgg = tf.contrib.slim.nets.vgg

In [ ]:
tf.reset_default_graph()
train_dir = '/tmp/loser_trail/'
dataset_dir = 'whimh2_vali_png.tfrecords'
num_samples = 1560 

with tf.device('/cpu:0'):
    #load data from dataset
    whimh2_train_png =Create_Slim_DateSet(dataset_dir,num_samples)
    images,labels = load_batch(whimh2_train_png, batch_size=16)
    image_large=tf.image.resize_images(images,[224,224])

    #concate vgg
    with slim.arg_scope(vgg.vgg_arg_scope()):
        predictions, _ =vgg.vgg_16(image_large,5)

    predictions = tf.argmax(predictions, 1)
    labels = tf.squeeze(labels)

#     summary_ops = []
#     accuracy = tf.contrib.metrics.accuracy(labels, predictions, name='preds0')
#     op = tf.summary.scalar('accuray', accuracy)
#     summary_ops.append(op)
    
    #Choose the metrics to compute:
    names_to_values, names_to_updates = slim.metrics.aggregate_metric_map({
        'accuracy': slim.metrics.streaming_accuracy(predictions, labels),
    })
    summary_ops=[]
    #Create the summary ops such that they also print out to std output:
    for metric_name, metric_value in names_to_values.iteritems():
        op = tf.summary.scalar(metric_name, metric_value)
        op = tf.Print(op, [metric_value], metric_name)
        summary_ops.append(op)

    num_examples = 10000
    batch_size = 32

    # Setup the global step.
    slim.get_or_create_global_step()

    output_dir = '/tmp/loser_trail/' # Where the summaries are stored.
    eval_interval_secs = 150 # How often to run the evaluation.

    slim.evaluation.evaluation_loop(
        '',
        '/tmp/loser_trail/',
        '/tmp/loser_trail/',
        num_evals=50,
        eval_op=names_to_updates.values(),
        summary_op=tf.summary.merge(summary_ops),
        eval_interval_secs=eval_interval_secs)


    #session_configuration 

INFO:tensorflow:Waiting for new checkpoint at /tmp/loser_trail/
INFO:tensorflow:Found new checkpoint at /tmp/loser_trail/model.ckpt-9000
INFO:tensorflow:Restoring parameters from /tmp/loser_trail/model.ckpt-9000
INFO:tensorflow:Starting evaluation at 2017-06-14-03:06:03
INFO:tensorflow:Evaluation [1/50]
INFO:tensorflow:Evaluation [2/50]
INFO:tensorflow:Evaluation [3/50]
INFO:tensorflow:Evaluation [4/50]
INFO:tensorflow:Evaluation [5/50]
INFO:tensorflow:Evaluation [6/50]
INFO:tensorflow:Evaluation [7/50]
INFO:tensorflow:Evaluation [8/50]
INFO:tensorflow:Evaluation [9/50]
INFO:tensorflow:Evaluation [10/50]
INFO:tensorflow:Evaluation [11/50]
INFO:tensorflow:Evaluation [12/50]
INFO:tensorflow:Evaluation [13/50]
INFO:tensorflow:Evaluation [14/50]
INFO:tensorflow:Evaluation [15/50]
INFO:tensorflow:Evaluation [16/50]
INFO:tensorflow:Evaluation [17/50]
INFO:tensorflow:Evaluation [18/50]
INFO:tensorflow:Evaluation [19/50]
INFO:tensorflow:Evaluation [20/50]
INFO:tensorflow:Evaluation [21/50]
INF

In [6]:
print labels
print predictions

Tensor("Squeeze:0", shape=(16,), dtype=int64, device=/device:CPU:0)
Tensor("ArgMax:0", shape=(16,), dtype=int64, device=/device:CPU:0)
